## 1. Library Imports

In [1]:
from pathlib import Path
import joblib  # Changed from pickle
import numpy as np
import tensorflow as tf
from PIL import Image
import keras

## 2. Configuration

In [2]:
PROJECT_ROOT = Path.cwd().parents[2]

MODEL_DIR = (
    PROJECT_ROOT
    / "agents"
    / "image_process"
    / "data"
    / "outputs"
    / "models"
)

LOGREG_PATH = MODEL_DIR / "logreg_derm.pkl"
DERM_MODEL_PATH = (
    MODEL_DIR
    / "models--google--derm-foundation"
    / "snapshots"
    / "a16a6ab4f87888948fe248136e697ed28146a1c6"
)

CLASS_NAMES = [
    "Eczema_Dermatitis",
    "Bacterial_Infections",
    "Fungal_Infections",
    "Viral_Infections",
    "Infestations",
    "Acneiform",
    "Vascular_Benign",
    "Healthy_Skin"
]

## 3. Load Models

In [3]:
# Load Derm Foundation model
derm_layer = keras.layers.TFSMLayer(
    str(DERM_MODEL_PATH),
    call_endpoint="serving_default"
)

derm_model = keras.Sequential([derm_layer])
print("Derm Foundation model loaded successfully")

Derm Foundation model loaded successfully


In [4]:
# Load Logistic Regression classifier
logreg = joblib.load(LOGREG_PATH)  

print("Logistic Regression classifier loaded successfully")

Logistic Regression classifier loaded successfully


## 4. Inference Functions

In [5]:
def encode_image_tf(image_path: str):
    """
    Extract embedding from image using Derm Foundation model
    """
    image_bytes = tf.io.read_file(image_path)
    
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "image/encoded": tf.train.Feature(
                    bytes_list=tf.train.BytesList(
                        value=[image_bytes.numpy()]
                    )
                )
            }
        )
    )
    
    serialized = example.SerializeToString()
    output_derm = derm_layer(inputs=tf.constant([serialized]))
    
    embedding = output_derm["embedding"].numpy().squeeze()
    return embedding

In [6]:
def predict_image(image_path: str):
    """
    Predict class for a single image
    """
    # Extract embedding
    embedding = encode_image_tf(image_path)
    
    # Reshape for sklearn
    embedding_reshaped = embedding.reshape(1, -1)
    
    # Predict class
    pred_class = logreg.predict(embedding_reshaped)[0]
    pred_proba = logreg.predict_proba(embedding_reshaped)[0]
    
    result = {
        "class_id": int(pred_class),
        "class_name": CLASS_NAMES[pred_class],
        "confidence": float(pred_proba[pred_class]),
        "all_probabilities": {
            CLASS_NAMES[i]: float(prob) 
            for i, prob in enumerate(pred_proba)
        }
    }
    
    return result

## 5. Example Usage

In [ ]:
TEST_IMAGE_PATH = "medscreening\\agents\\image_process\\data\\pediatric_skin_data\\4_Infestations\\0_PA-cutaneous-larva-migrans (40).jpg"

result = predict_image(TEST_IMAGE_PATH)

print(f"\nPrediction Results:")
print(f"Class: {result['class_name']}")
print(f"Confidence: {result['confidence']:.2%}")
print(f"\nAll Class Probabilities:")
for class_name, prob in result['all_probabilities'].items():
    print(f"{class_name}: {prob:.2%}")


Prediction Results:
Class: Infestations
Confidence: 99.90%

All Class Probabilities:
Eczema_Dermatitis: 0.03%
Bacterial_Infections: 0.00%
Fungal_Infections: 0.06%
Viral_Infections: 0.00%
Infestations: 99.90%
Acneiform: 0.00%
Vascular_Benign: 0.00%
Healthy_Skin: 0.00%
